 **Content**

1. [Introduction](#Introduction)

2. [The following are the important libraries](#The-following-are-the-important-libraries)

   2.1. [Correct Data for Comparison](#Correct-Data-for-Comparison)
   
   2.2. [Reading JSON Data](#Reading-JSON-Data)
   
3. [Dashboard for Predictive Maintenance](#Dashboard-for-Predictive-Maintenance)

   3.1. [Initialization](#Initialization)
   
   3.2. [File Upload Section](#File-Upload-Section)
   
   3.3. [Data Processing](#Data-Processing)
   
   3.4. [Data Display](#Data-Display)
   
   3.5. [Test Experiment](#Test-Experiment)
   
   3.6. [Data Drift Experiment](#Data-Drift-Experiment)
   
   3.7. [Confusion Matrix](#Confusion-Matrix)
   
   3.8. [Correlation Matrix](#Correlation-Matrix)
   
   3.9. [Error Handling](#Error-Handling)
   
   3.10. [Main Execution](#Main-Execution)


You can use these clickable section titles to navigate to specific sections in your notebook.

# Introduction
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

This notebook serves as a comprehensive Predictive Maintenance Dashboard, allowing users to interact with the results of predictive maintenance analyses. It utilizes a Streamlit web application to provide an intuitive and user-friendly interface for exploring machine health data and predictive models.

The primary objective of this dashboard is to enable users to:

- Upload a CSV file containing machine data.
- Process the uploaded data using a FastAPI backend.
- Visualize and analyze the results of predictive maintenance experiments.
- Explore machine health indicators and their impact.

The dashboard offers several features, including data visualization, experiment selection, and a "Run" button to trigger the analysis process. It allows users to choose different types of experiments, such as "Test," "Data Drift," "Confusion Matrix," and "Correlation Matrix," providing a versatile platform for predictive maintenance tasks.


# The following are the important libraries

............................................................................................................................................................................................................


In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import time
import os
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import requests
import json
from pathlib import Path
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time

## Correct Data for Comparison
This section loads and prepares the data for comparison and analysis. It consists of the following steps:

1. Reading and preprocessing the CSV data file.
2. Detecting peaks and valleys in the signal data.
3. Data filtering and segmentation.
4. Grouping the data into experiments based on detected peaks and valleys.
5. Filtering the grouped data based on specific conditions.


In [ ]:
corr_data_dir= "2020-11-25_09-10-56-DTT-MILLP500U-X.csv"

column_indices = [0, 1, 2, 4, 6, 8, 9]
expected_columns = ['Time','PosDiff/X', 'v act/X', 's act/X','a act/X', 'I (nom)/X', 's diff/X']
initial_time = -464151000.0

df_corr = pd.read_csv(corr_data_dir,delimiter=";", usecols=column_indices)
df_corr.columns= expected_columns
df_corr = df_corr[df_corr['Time'] >= initial_time].reset_index(drop=True)



peaks, _ = find_peaks(df_corr.iloc[:,3])
#peaks_idx.append(peaks)
valleys, _ = find_peaks(-df_corr.iloc[:,3])  # Find valleys by negating the signal data
#valleys_idx.append(valleys)

a= len(df_corr.iloc[:,3][valleys][(df_corr.iloc[:,3][valleys] > 50) & (df_corr.iloc[:,3][valleys] < 600) & (df_corr.iloc[:,3][valleys].index<20000)].index)
b= len(df_corr.iloc[:,3][peaks][(df_corr.iloc[:,3][peaks] > 500) & (df_corr.iloc[:,3][peaks] < 600) & (df_corr.iloc[:,3][peaks].index>20000) & (df_corr.iloc[:,3][peaks].index<80000)].index) 

warm_len= a + b

start_point = warm_len
end_point = peaks.shape[0]

# Calculate the step size based on the number of steps (3 in this case)
step_size = 3
# Create the array with multiples of 3 till 15 times
req_segment= np.arange(warm_len, end_point,step_size)


exp_corr = []

for j in range(0,5):
    exp_corr.append(pd.DataFrame(df_corr[valleys[req_segment[j]]: valleys[req_segment[j+1]]]).reset_index(drop= True))
for j in range(5,10):
    exp_corr.append(pd.DataFrame(df_corr[valleys[req_segment[j]]: valleys[req_segment[j+1]]]).reset_index(drop=True))
for j  in range(10,15):
    exp_corr.append(pd.DataFrame(df_corr[peaks[req_segment[j]]: peaks[req_segment[j+1]]]).reset_index(drop=True))


exp_corr1_1= (exp_corr[0][exp_corr[0].iloc[:,3]> 332].reset_index(drop= True))
exp_corr1_2= (exp_corr[1][exp_corr[1].iloc[:,3]>323])
exp_corr1_3= (exp_corr[2][exp_corr[2].iloc[:,3]>310])
exp_corr1_4= (exp_corr[3])
exp_corr1_5= (exp_corr[4][exp_corr[4].iloc[:,3]>250])

exp_corr_1 = [exp_corr1_1, exp_corr1_2, exp_corr1_3, exp_corr1_4, exp_corr1_5] 

exp_corr_2= []
exp_corr_3= []

for i in range(5,15):
    if i < 10:
        exp_corr_2.append(exp_corr[i])
    else :
        exp_corr_3.append(exp_corr[i])

experiments_corr= [exp_corr_1,exp_corr_2,exp_corr_3]



## Reading JSON Data
This section covers the process of reading JSON data from files. It includes the following steps:

1. Defining a function to read JSON files.
2. Specifying the root directory where JSON files are stored.
3. Iterating through files and directories to collect JSON data.
4. Sorting the collected files based on location and file name.
5. Creating a list of defective indexes.
6. Reading and collecting experiments from JSON files.

The provided code structures and processes the data for further analysis and comparison.


In [ ]:
def read_json_file(file_path):
    try:
        data = pd.read_json(file_path).reset_index(drop=True)
        return data
    except Exception as e:
        st.error(f"An error occurred while reading JSON data from {file_path}: {str(e)}")
        return None

rootdir = "/GM_usecase/backend_fastapi/app/faulty_data"

all_faults = []
location = []

for root, directories, files in os.walk(rootdir):
    for file_name in files:
        location.append(root)
        all_faults.append(file_name)

def sorting_location(path):
    # Split the path by '_' and get the last part
    parts = path.split('_')
    # Convert the last part to an integer for sorting
    return int(parts[-1])

def sorting_file_name(path):
    # Split the path by '_' and get the last part
    parts = (path.replace(".","_")).split("_")
    # Convert the last part to an integer for sorting
    return int(parts[-2])

# Sort the file_paths based on the end number
sorted_loc = sorted(location, key=sorting_location)
sorted_file_name = sorted(all_faults, key=sorting_file_name)

loc_faulty_signal = [os.path.join(x, y) for x, y in zip(sorted_loc, sorted_file_name)]

defective_index = []
experiments = []

for i, file_path in enumerate(loc_faulty_signal):
    defective_index.append(sorting_location(location[i]))
    defective_index = sorted(defective_index)
    data = read_json_file(file_path)
    if data is not None:
        experiments.append(data)

# Dashboard for Predictive Maintenance
This section sets up a Streamlit dashboard for predictive maintenance. It allows users to upload a CSV file, process it, and interact with the results. The code description includes the following:

## Initialization
- Initializes the Streamlit app and session state.
- Creates the main title and subheader for the dashboard.

## File Upload Section
- Allows users to upload a CSV file.
- Checks if a file is uploaded and shows a processing message.

## Data Processing
- Sends an HTTP POST request to a FastAPI endpoint to process the uploaded CSV file.
- Handles responses from FastAPI.
- Validates the response status code and displays relevant information.
- Parses the uploaded file name to extract machine type, date, and axis.
- Creates widgets for user interaction in the sidebar, including experiment selection, machine type, date, and axis.
- Provides a "Run" button to load new results.

## Data Display
- Displays the selected options, including experiment type.
- Provides a classification of defect severity.
- Allows users to select a signal type (e.g., 'PosDiff/X', 'v act/X') to display.

## Test Experiment
- If 'Test' experiment is selected, iterates through defective indexes and displays line charts for each.

## Data Drift Experiment
- If 'Data Drift' experiment is selected, parses and displays the response data as a DataFrame.

## Confusion Matrix
- If 'Confusion Matrix' is selected, displays an image of a confusion matrix.

## Correlation Matrix
- If 'Correlation Matrix' is selected, displays an image of a correlation matrix.

## Error Handling
- Provides error messages for any issues that occur during file upload, processing, or plotting.

## Main Execution
- The code execution begins when the script is run.

This code sets up a Streamlit dashboard for interacting with predictive maintenance results.


In [ ]:
def main():

    if 'run_button_state' not in st.session_state:
        st.session_state.run_button_state = False

    st.title('Dashboard')
    st.subheader('Predictive Maintenance')

    # File Upload Section
    uploaded_file = st.file_uploader("Choose a CSV file", type=["csv"])
    
    if uploaded_file is not None:
        st.write("Processing the uploaded file...")
    else:
        st.write("Upload a CSV file to process.")

    
    if uploaded_file is not None:

        try:

            # Add the retry mechanism
            retry_strategy = Retry(
                total=5,
                backoff_factor=2,
                status_forcelist=[429, 500, 502, 503, 504],
            )
            adapter = HTTPAdapter(max_retries=retry_strategy)
            http = requests.Session()
            http.mount("http://", adapter)

            # Make an HTTP POST request to your FastAPI endpoint
            api_endpoint = "http://fastapi:80/predict/"
            
            # Prepare the file for upload
            files = {'file': (uploaded_file)}
            
            response = requests.post(api_endpoint, files=files)
            
            if (response.status_code == 201) | (response.status_code==200) :
                #st.write("Response from FastAPI:")
                
                # Once the file upload and response are complete, you can display your charts here.
                # Create widgets for user interaction
                
                uploaded_file_name= uploaded_file.name

                substrings_to_check = ["MILLP500", "MillP500", "Mill_P500"]
                date = uploaded_file_name.split('_')[0]
                axis= uploaded_file_name.replace('.',"-").split('-')[-2]

                # Check if any of the substrings is present in the string
                found_substring = any(substring in uploaded_file_name for substring in substrings_to_check)

                if found_substring:
                    Machine_type= "MILL P 500 U"
                else:
                    Machine_type= 'None'
                
        
                # Create widgets for user interaction in the sidebar
                with st.sidebar:
                    experiment_selector = st.radio('Select Experiment', ['Test','Data Drift','Confusion Matrix',"Correlation Matrix"])
                    selected_machine_type = st.radio('Machine Type', [Machine_type])
                    selected_date = st.radio('Date', [date])
                    selected_axis = st.radio('Axis', [axis])
                    st.write("NOTE: Please Share The 'Feedback'")
                    
                ### run button to load result
                run_button = st.button('Run')
                new_title = '<p style="font-family:sans-serif; color:Black; font-size: 12px;">Please Press "RUN" to load the new results</p>'
                st.markdown(new_title, unsafe_allow_html=True)

                # Display the selected options
                st.write("Type:", experiment_selector)
                
                columns = {'PosDiff/X': 1, 'v act/X': 2, 's act/X': 3, 'a act/X': 4, 'I (nom)/X': 5, 's diff/X': 6}


                # Loop through defective indexes and display plots based on user selections
                if experiment_selector == 'Test':
                    def classify_defect(x):
                        if x <= 4:
                            return "Low Critical"
                        elif 5 <= x < 10:
                            return "Critical"
                        else:
                            return "High Critical"
                    
                    st.subheader(f'The total number of Defect: {len(defective_index)} and Health Indicator: {classify_defect(len(defective_index))}')
                        
                    selected_col = st.selectbox('Signal Type', columns.keys())

                    for i, indx in enumerate(defective_index):
                        try:
                            data = experiments[i].iloc[:, columns[selected_col]]
                            data1 = experiments_corr[indx // 5][indx % 5].iloc[:, columns[selected_col]]

                            # Create a DataFrame for this iteration's data
                            df = pd.DataFrame({'Test': data, 'Corr': data1})

                            # Display the plot for this iteration
                            st.subheader(f'Defective Index {indx}, Experiment {experiment_selector}')
                            st.line_chart(df, use_container_width=True)
                        except Exception as e:
                            st.error(f"An error occurred while plotting: {str(e)}")

                elif experiment_selector == 'Data Drift':
                        response_dict = json.loads(response.text)

                        # Now response_dict is a Python dictionary
                        st.write(pd.DataFrame(list(response_dict.items())[1][1]))
                    
                elif experiment_selector == 'Confusion Matrix':
                    # Your image file path
                    image_path = f"{BASE_DIR}/Confusion_Matrix.png"
                    st.image(image_path,channels="BGR")

                else :
                    # Your image file path
                    image_path = f"{BASE_DIR}/correlation_matrix.png"
                    st.image(image_path,channels="BGR")

                
            else:
                st.error(f"Failed to upload file. Status code: {response.status_code}")
        except Exception as e:
            st.error(f"An error occurred while uploading the file: {str(e)}")

if __name__ == "__main__":
    main()